In [1]:
# Importy
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from typing import Callable
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Pobranie danych
heart_disease = fetch_ucirepo(id=45)

# Porzucenie linii z pustymi etykietami oraz odpowiadajacych im wartosci
feature_matrix = heart_disease.data.features.dropna()
labels = heart_disease.data.targets.loc[feature_matrix.index]

# Przetworzenie zbioru wartości przewidywanych do wartości binarnych
y_binary = labels.copy()
y_binary['num'] = y_binary['num'].apply(lambda x: 1 if x != 0 else 0)                      

# Utworznnie zbioru dummy etykiet
x_dummy = pd.get_dummies(feature_matrix, columns=['cp', 'restecg', 'slope','ca','thal'])   

# upewnienie się co do typów wykorzystywanych danych
x_dummy = x_dummy.fillna(0).astype(float)
y_binary = y_binary.astype(float)

# Podział danych na zbiór uczący i testowy
x_train, x_test, y_train, y_test = train_test_split(x_dummy, y_binary, test_size=0.2, random_state=268555)

# Konwersja danych do tablic NumPy przed utworzeniem tensorów PyTorch
X_train_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)  # wektor kolumnowy
X_test_tensor = torch.tensor(x_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)    # wektor kolumnowy

# # Normalizacja cech trenignowych i testowych
# scaler = StandardScaler()
# x_train_normalised = scaler.fit_transform(x_train)
# x_test_normalised = scaler.transform(x_test)

# # Konwersja danych do wymaganego formatu
# x_train = np.array(x_train).astype(float)
# y_train = np.array(y_train).astype(float)

# x_test = np.array(x_test).astype(float)
# y_test = np.array(y_test).astype(float)
 

In [26]:
# Stworzona klasa Dataset
class HeartDiseaseDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Implementacja klasy DataLoader
train_dataset = HeartDiseaseDataset(X_train_tensor, y_train_tensor)
test_dataset = HeartDiseaseDataset(X_test_tensor, y_test_tensor)

# Stworzony DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Stworzony model sieci neuronowej (jedna warstwa ukryta, 16 perceptonów)
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=16):
        super(HeartDiseaseModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Model instantiation
input_dim = x_train.shape[1]
model = HeartDiseaseModel(input_dim)

# Step 4: Define loss function and optimizers
criterion = nn.BCELoss()
optimizers = {
    "SGD": optim.SGD(model.parameters(), lr=0.05),
    "Adam": optim.Adam(model.parameters(), lr=0.05),
    "RMSprop": optim.RMSprop(model.parameters(), lr=0.05)
}

# Training loop function
def train_model(model, optimizer, num_epochs=30):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        for inputs, labels in train_loader:
            labels = labels.squeeze(1)  # Usunięcie dodatkowego wymiaru
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        # Print epoch loss
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

# Evaluate the model
def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            preds = outputs.round()  # Rounding to get binary predictions
            all_preds.extend(preds.numpy().flatten())
            all_labels.extend(labels.numpy().flatten())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    return accuracy, f1, precision

# Step 5: Train and evaluate using different optimizers
results = {}
for name, optimizer in optimizers.items():
    print(f"\nTraining with {name} optimizer:")
    model = HeartDiseaseModel(input_dim)  # Reinitialize the model for each optimizer
    train_model(model, optimizer)
    accuracy, f1, precision = evaluate_model(model, test_loader)
    results[name] = {
        "accuracy": accuracy,
        "f1_score": f1,
        "precision": precision
    }

results


Training with SGD optimizer:
Epoch [1/30], Loss: 40.6470
Epoch [2/30], Loss: 40.3031
Epoch [3/30], Loss: 40.2808
Epoch [4/30], Loss: 40.4085
Epoch [5/30], Loss: 39.7398
Epoch [6/30], Loss: 39.9454
Epoch [7/30], Loss: 40.8829
Epoch [8/30], Loss: 39.7282
Epoch [9/30], Loss: 39.9934
Epoch [10/30], Loss: 40.8886
Epoch [11/30], Loss: 40.7549
Epoch [12/30], Loss: 40.2057
Epoch [13/30], Loss: 39.6110
Epoch [14/30], Loss: 40.7238
Epoch [15/30], Loss: 40.2583
Epoch [16/30], Loss: 39.5244
Epoch [17/30], Loss: 39.7376
Epoch [18/30], Loss: 40.3023
Epoch [19/30], Loss: 39.9521
Epoch [20/30], Loss: 39.9752
Epoch [21/30], Loss: 40.7056
Epoch [22/30], Loss: 40.2508
Epoch [23/30], Loss: 40.0660
Epoch [24/30], Loss: 40.6838
Epoch [25/30], Loss: 40.7300
Epoch [26/30], Loss: 39.7248
Epoch [27/30], Loss: 40.5340
Epoch [28/30], Loss: 40.2245
Epoch [29/30], Loss: 40.8285
Epoch [30/30], Loss: 40.8254

Training with Adam optimizer:
Epoch [1/30], Loss: 5.4755
Epoch [2/30], Loss: 5.5371
Epoch [3/30], Loss: 5.48

{'SGD': {'accuracy': 0.4166666666666667,
  'f1_score': np.float64(0.5882352941176471),
  'precision': np.float64(0.4166666666666667)},
 'Adam': {'accuracy': 0.4166666666666667,
  'f1_score': np.float64(0.5882352941176471),
  'precision': np.float64(0.4166666666666667)},
 'RMSprop': {'accuracy': 0.4166666666666667,
  'f1_score': np.float64(0.5882352941176471),
  'precision': np.float64(0.4166666666666667)}}